In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import lightgbm as lgb

from hyperopt import hp, tpe, fmin, Trials
from hyperopt.pyll.base import scope

import mlflow
import mlflow.lightgbm

import sys
import os
sys.path.append('/Users/ben/Desktop/py_proj/account_funds_prediction')
sys.path.append('/Users/ben/Desktop/py_proj/account_funds_prediction/src')

#from src.lightgbm_model_v3 import lightgbm_dev
from src.lightgbm_model import lightgbm_dev_v2
from utils.metrics import PowerRatio

import warnings
warnings.filterwarnings("ignore")

In [2]:
sys.path

['/Library/Frameworks/Python.framework/Versions/3.9/lib/python39.zip',
 '/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9',
 '/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/lib-dynload',
 '',
 '/Users/ben/Desktop/py_proj/account_funds_prediction/venv1/lib/python3.9/site-packages',
 '/Users/ben/Desktop/py_proj/account_funds_prediction',
 '/Users/ben/Desktop/py_proj/account_funds_prediction/src']

In [3]:
os.getcwd()

'/Users/ben/Desktop/py_proj/account_funds_prediction/notebook'

## 1. Data prep

In [4]:
df = pd.read_csv('/Users/ben/Desktop/py_proj/account_funds_prediction/data/df_v2')

In [5]:
df.head()

,bal_now,age,tenure,credit_score,annual_income,inflow,outflow,mtg_balance,credit_card_balance,loan_balance,...,fea27,fea28,fea29,fea30,fea31,fea32,fea33,fea34,fea35,bal_after_6mon
0,6423388,27,24,837,191218,19428,1685,758311,15470,68750,...,48203,40848,17735,92914,7025,38486,43083,14451,68160,6.320044e+06
1,6550634,22,43,585,239710,5914,7998,813964,7092,458988,...,13815,4393,90459,8199,21066,29089,46831,4332,24124,6.418258e+06
2,4304572,47,25,616,127094,9698,8166,825144,33306,168257,...,22362,12895,48617,64084,20536,61102,10177,88745,53813,4.173464e+06
3,2234489,65,2,497,79674,17023,2248,74580,4987,64865,...,18514,79026,51272,6730,71238,74501,57498,45620,55350,2.262496e+06
4,9958614,38,34,623,132976,810,4576,7523,1726,252167,...,55122,98179,49028,55157,51188,52524,75274,18425,83985,9.996288e+06


In [7]:
X = df.drop(columns=['bal_after_6mon'])
y = df['bal_after_6mon']

In [8]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

## 2. Model Training

### 2.1 Baseline model

In [9]:
baseline_params = {
    'max_depth': 3,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'feature_fraction': 0.5
}

search_space = {
    # Integer parameters
    'max_depth': scope.int(hp.quniform('max_depth', 3,10,1)),
    'n_estimators': scope.int(hp.quniform('n_estimators', 50,300,50)),
    'num_leaves': scope.int(hp.quniform('num_leaves', 20, 100, 5)),  
    'min_data_in_leaf': scope.int(hp.quniform('min_data_in_leaf', 5, 100, 5)),  

    # Decimal parameters
    'learning_rate': hp.uniform('learning_rate', 0.01, 1),  # Typical range for learning rate
    'feature_fraction': hp.uniform('feature_fraction', 0.3, 1.0),  # Typical range for feature_fraction
    'bagging_fraction': hp.uniform('bagging_fraction', 0.3, 1.0),  # Typical range for bagging_fraction
    'lambda_l1': hp.uniform('lambda_l1', 0, 1),  # Typical range for lambda_l1 (L1 regularization)
}


In [10]:
lgb_dev1 = lightgbm_dev_v2(X_train=X_train, y_train=y_train, 
                   X_test=X_test, y_test=y_test,
                   baseline_params = baseline_params,
                   baseline_ind=1,
                   max_evals=2,
                   search_space=search_space)

In [11]:
lgb_dev1

In [12]:
lgb1 = lgb_dev1.run_model()

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010614 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11592
[LightGBM] [Info] Number of data points in the train set: 140000, number of used features: 50
[LightGBM] [Info] Start training from score 4906522.606439
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5


2024/12/28 16:06:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


MLflow Run ID: a59da3af62d547a89f67c9554ea72175


In [13]:
lgb1

(LGBMRegressor(feature_fraction=0.5, max_depth=3, n_estimators=50),
 'a59da3af62d547a89f67c9554ea72175')

In [14]:
lgb1_id = lgb1[1]
lgb1_id

'a59da3af62d547a89f67c9554ea72175'

In [15]:
# Fetch the run details using MLflow API
run = mlflow.get_run(lgb1_id)

# Retrieve the metrics from the run
rmse_train = run.data.metrics.get("rmse_train", None)
rmse_test = run.data.metrics.get("rmse_test", None)
r2_train = run.data.metrics.get("r2_train", None)
r2_test = run.data.metrics.get("r2_test", None)
pr_train = run.data.metrics.get("powerratio_train", None)
pr_test = run.data.metrics.get("powerratio_test", None)

# Print the retrieved metrics
print(f"RMSE Train: {rmse_train}")
print(f"RMSE Test: {rmse_test}")
print(f"R2 Train: {r2_train}")
print(f"R2 Test: {r2_test}")
print(f"Power Ratio Train: {pr_train}")
print(f"Power Ratio Test: {pr_test}")

RMSE Train: 270882.6128560889
RMSE Test: 272238.4425419598
R2 Train: 0.9911632315521512
R2 Test: 0.9910722366311477
Power Ratio Train: 0.9999533628349826
Power Ratio Test: 0.9999527159450317


### 2.2 HyperOpt (50 fits)

In [16]:
lgb_dev2 = lightgbm_dev_v2(X_train=X_train, y_train=y_train, 
                   X_test=X_test, y_test=y_test,
                   baseline_params = baseline_params,
                   baseline_ind=0,
                   max_evals=50,
                   search_space=search_space)

In [17]:
%%time
lgb2 = lgb_dev2.run_model()

[LightGBM] [Warning] lambda_l1 is set=0.9112715004133979, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9112715004133979            
[LightGBM] [Warning] feature_fraction is set=0.6937921859364116, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6937921859364116
[LightGBM] [Warning] bagging_fraction is set=0.8458670925144631, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8458670925144631
[LightGBM] [Warning] min_data_in_leaf is set=55, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=55                       
[LightGBM] [Warning] lambda_l1 is set=0.9112715004133979, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9112715004133979            
[LightGBM] [Warning] feature_fraction is set=0.6937921859364116, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6937921859364116
[LightGBM] [Warning] bagging_fraction is set=0.8458670925144631, subsample=1.0 will be ignored. Current value:

2024/12/28 16:09:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


MLflow Run ID: 829458b387ef42b4888a51822580f459
best param: {'num_leaves': 20, 'max_depth': 10, 'n_estimators': 300, 'min_data_in_leaf': 100, 'learning_rate': 0.23343967842553467, 'feature_fraction': 0.8830460359876731, 'bagging_fraction': 0.43060883678575035, 'lambda_l1': 0.7435503912310693}
CPU times: user 11min 55s, sys: 2min 43s, total: 14min 38s
Wall time: 2min 10s


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM]

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM]

2024/12/26 13:49:03 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during lightgbm autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 7acde082c46148dbb0f568f9e9350173. Failed operations: [MlflowException("Changing param values is not allowed. Param with key=\'learning_rate\' was already logged with value=\'0.730236549985059\' for run ID=\'7acde082c46148dbb0f568f9e9350173\'. Attempted logging new value \'0.8769880053165809\'.")]')]



[LightGBM] [Warning] lambda_l1 is set=0.2570035237866054, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2570035237866054                                
[LightGBM] [Warning] feature_fraction is set=0.4460491275969355, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4460491275969355           
[LightGBM] [Warning] bagging_fraction is set=0.8704779571454411, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8704779571454411                  
[LightGBM] [Warning] min_data_in_leaf is set=95, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=95                                           
[LightGBM] [Warning] lambda_l1 is set=0.2570035237866054, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2570035237866054                                
[LightGBM] [Warning] feature_fraction is set=0.4460491275969355, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4460491275969355           
[LightGBM]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [18]:
lgb2

(LGBMRegressor(bagging_fraction=0.43060883678575035,
               feature_fraction=0.8830460359876731, lambda_l1=0.7435503912310693,
               learning_rate=0.23343967842553467, max_depth=10,
               min_data_in_leaf=100, n_estimators=300, num_leaves=20),
 '829458b387ef42b4888a51822580f459')

In [20]:
# Fetch the run details using MLflow API
run_id = lgb2[1]
run = mlflow.get_run(run_id)

# Retrieve the metrics from the run
rmse_train = run.data.metrics.get("rmse_train", None)
rmse_test = run.data.metrics.get("rmse_test", None)
r2_train = run.data.metrics.get("r2_train", None)
r2_test = run.data.metrics.get("r2_test", None)
pr_train = run.data.metrics.get("powerratio_train", None)
pr_test = run.data.metrics.get("powerratio_test", None)

# Print the retrieved metrics
print(f"RMSE Train: {rmse_train}")
print(f"RMSE Test: {rmse_test}")
print(f"R2 Train: {r2_train}")
print(f"R2 Test: {r2_test}")
print(f"Power Ratio Train: {pr_train}")
print(f"Power Ratio Test: {pr_test}")

RMSE Train: 12113.402091569065
RMSE Test: 12812.746794690169
R2 Train: 0.9999823289041604
R2 Test: 0.9999802244643838
Power Ratio Train: 0.9999956238937182
Power Ratio Test: 0.9999951161887085
